In [7]:
# 필요한 module import
import os
import time
import glob

from PyPDF2 import PdfFileReader, PdfFileWriter
from pdfminer.layout import LAParams, LTTextLine, LTTextBox
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator

from pdf2image import convert_from_path, convert_from_bytes
from IPython.display import display
from PIL import Image, ImageDraw

#conver index(int) to str of specific size ex. 4 -> '0004' 
def index_convert_filename_regular_size(index, size):
    idx_len = len(str(index))
    filename = ''
    for _ in range(size-idx_len):
        filename += '0'
    filename += str(index)
    return filename    

# intial setting to progress PDF-_crop_function
def initialize(file_path):
    try:
        os.mkdir(file_path+'/pdf')
    except:
        print("already exist folder", file_path+'/pdf')
    try:
        os.mkdir(file_path+'/output')
    except:
        print("already exist folder", file_path+'/output')

    print('plz put in pdf files at '+ file_path +'/pdf')

    
    
#description-----------------------------------------------------    
#pdf_file : name of pdf file in pdf folder ex.'asd.pdf' 
#file_path : current dir path ex. os.getcwd()
#output_format : yolo(x_center, y_center, width, height, value), ocr(filename, value) 
#time_check(default False) : if you want to check time use this 
#save_crop_img(default True) : decide to save crop image 
#single_page : need update
def PDF_crop_function(pdf_file ,file_path, output_format,page_range = None, time_check = 0, save_crop_img = 1, save_label = 1, a = 0, print_option = False, draw_option = False):    
    if time_check ==1:
        start = time.time()
    #path setting
    pdf_path = file_path + '/pdf/' + pdf_file
    save_path = file_path +'/output/' + pdf_file

    #make dir to save
    try:
        os.mkdir(save_path)
    except:
        print("already exist folder", save_path)
    #convert pdf to PIL image list
    
    print("converting pdf to image...")
    img = convert_from_path(pdf_path)
    page_num = len(img)
    
    print("getting pdf data ...")
    fp = open(pdf_path, 'rb')
    # pdf coordinate extraction 하기 위해서 필요한 변수 setting
    rsrcmgr = PDFResourceManager()
    laparams = LAParams(detect_vertical = True)
    device = PDFPageAggregator(rsrcmgr, laparams=laparams)
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    pages = PDFPage.get_pages(fp)

    i = 0 # cnt for progress page
    check = 0 # ckeck for progress
    if page_range == None:
        page_range = range(page_num) 
    else:
        page_range = range(page_range[0], page_range[1]) 
     
    for page in pages:
        #pregress print
        if i/page_num > 0.25 and check == 0:
            print('25% complete,', page_num-i, 'page is remain...')
            check = 1
        elif i/page_num > 0.5 and check == 1:
            print('50% complete,', page_num-i, 'page is remain...')
            check = 2
        elif i/page_num > 0.75 and check == 2:
            print('75% complete,', page_num-i, 'page is remain...')
            check = 3
     
        
        #get data og the pdf page of i
        if  i in page_range: #only print single page
            x_max = page.mediabox[2]
            y_max =  page.mediabox[3]
            img_size = img[i].size
            page_path = save_path + '/' +index_convert_filename_regular_size(i, 4)
            try:
                os.mkdir(page_path)
            except: 
                #print("already exist folder", page_path)
                pass
            
            interpreter.process_page(page)
            layout = device.get_result()
            with open(page_path +'/labels.txt', "w") as f:
                j = 0
                for obj in layout:
                    if isinstance(obj,LTTextBox):
                        for lobj in obj:
                            if isinstance(lobj,LTTextLine):
                                x1, x2, y1, y2, value = lobj.bbox[0], lobj.bbox[2], lobj.bbox[1], lobj.bbox[3], lobj.get_text()
                                #PIL coordinate transform
                                #print('x1:', x1,'x2:', x2, 'y1:', y1,'y2:', y2, value)
                                x1 = x1/(x_max)*img_size[0]
                                x2 = x2/(x_max)*img_size[0]
                                y1 = (1 - (y1/(y_max - a)))*img_size[1]
                                y2 = (1 - (y2/(y_max - a)))*img_size[1]
                                if draw_option == True:
                                    draw = ImageDraw.Draw(img[i])
                                    draw.rectangle([x1, y1,x2, y2], outline='yellow', width = 1)
                               
                                #write labeling data at txt. ex. bbox cordinate, value 
                                if save_label == True:
                                    if output_format == 'yolo':
                                        x_center = (x1 + x2)/2
                                        y_center = (y2 + y1)/2
                                        width = x2-x1
                                        height = y1-y2
                                        f.write("0 %f %f %f %f %s" % (x_center, y_center, width, height, value))  
                                    elif output_format == 'ocr':
                                        f.write("%s\t%s" % (index_convert_filename_regular_size(j, 4)+'.jpg', value))  
                                
                                #save the crop image
                                if save_crop_img == True:
                                    crop_img = img[i].crop((x1, y2, x2, y1))
                                    if 2*abs(x1-x2) < abs(y1-y2):
                                        crop_img = crop_img.rotate(90, expand=True)
                                    crop_img.save(page_path+ '/'+index_convert_filename_regular_size(j, 4)+'.jpg')
                                    if print_option ==True: display(crop_img)
                                    if print_option ==True: print(value)
                                j += 1
            if print_option ==True: display(img[i])
        i += 1
    if time_check ==1:
        print(time.time()-start)
    print('100% complete')
       

In [6]:
file_path = os.getcwd()
initialize(file_path)

plz put in pdf files at /mnt/pdf_crop/pdf


In [ ]:
pdf_file = 'sample1.pdf'
file_path = os.getcwd()
output_format = 'ocr'
time_check = 1
page_range = None
PDF_crop_function(
    pdf_file,file_path, 
    output_format, 
    page_range,
    print_option = False,
    save_crop_img= True, 
    draw_option=False, 
    save_label= True, 
    a=0
)

In [ ]:
pdf_file = 'sample3.pdf'
dic_list = os.listdir(file_path+'/output/'+pdf_file)
dic_list.sort()
for i in range(40):
    file_list = glob.glob(file_path+'/output/'+pdf_file+'/'+dic_list[i]+'/*.jpg')
    file_list.sort()
    with open(file_path+'/output/'+pdf_file+'/'+dic_list[i]+'/Labels.txt') as f:
        lines = f.readlines()
    for j in range(len(file_list)):
        display(Image.open(file_list[j]))
        print(lines[j])

In [4]:
pdf_list = os.listdir("./pdf")
output_list = os.listdir("./output")
pdf_list = list(set(pdf_list) - set(output_list)) 
pdf_list[:5]

['._1225-9071_2003_v20n2_85.pdf',
 '1226-3133_2000_동심유전체봉이삽입된원통형공진기의정확한전자계표현.pdf',
 '1226-3133_2014_거친도체표면후방산란계산을위한모서리효과저감기법.pdf',
 '._1226-3133_2016_v27n12_1084.pdf',
 '._1226-3133_2004_PBG구조를이용한DualBand전력증폭기연구.pdf']

In [ ]:

check = 0
for i in range(len(pdf_list)):
    print(pdf_list[i],'progress...')
    if i/len(pdf_list) > 0.25 and check == 0:
        print('25% complete,', len(pdf_list)-i, 'pdf is remain-----------------------------------')
        check = 1
    elif i/len(pdf_list) > 0.5 and check == 1:
        print('50% complete,', len(pdf_list)-i, 'pdf is remain-----------------------------------')
        check = 2
    elif i/len(pdf_list)> 0.75 and check == 2:
        print('75% complete,', len(pdf_list)-i, 'pdf is remain-----------------------------------')
        check = 3


    PDF_crop_function(
        pdf_file = pdf_list[i],
        file_path = os.getcwd(), 
        output_format ='ocr', 
        page_range = None,
        print_option = False,
        save_crop_img= True, 
        draw_option=False, 
        save_label= True, 
        a=0
    )

In [11]:
pdf_list = os.listdir("./pdf")
output_list = os.listdir("./output")
pdf_list = list(set(pdf_list) - set(output_list)) 
print(pdf_list[:5])
print('total', len(pdf_list), 'completed :', len(output_list))

['._1225-6692_2004_v25n2_109.pdf', '1598-706X_@nondate_주형에서석탄분첨가제시콜에의한주철주물의소착억제기구에관하여.pdf', '0367-6315_@nondate_PersistenceofSalmonellaentericaEscherichiacoliO157H7andListeriamonocytogenesinSoilLiquidManureAmendedSoilandLiquidManure.pdf', '._0367-6315_@nondate_시설가지연작재배토양이화학성이생리장해과발생에미치는영향.pdf', '._2288-6559_@nondate_알칼리금속이흡착된Si111TEX7times7TEX계의초기산화과정연구.pdf']
total 17578 completed : 17576


In [10]:
from function import PDF_crop_function
import multiprocessing as mp
from itertools import repeat

pool = mp.Pool(processes=32)
pool.map(PDF_crop_function,pdf_list)
pool.close()
pool.join()


already exist folder /mnt/pdf_crop/output/
already exist folder /mnt/pdf_crop/output/


In [31]:
pdf_dir_list = glob.glob('./outputs/*/*.pdf')


In [32]:
len(pdf_dir_list)

11451

In [35]:
import shutil

In [38]:
for pdf_fn in pdf_dir_list:
    dst_fn = pdf_fn[:pdf_fn.rfind('.')]
    shutil.move(pdf_fn, dst_fn)